<a href="https://colab.research.google.com/github/chaithanya21/Word-Embedding-In-Tensorflow/blob/master/Word_Embeddings_In_Tensorflow_On_IMDB_Movie(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0

In [26]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
!pip install -q tensorflow-datasets

In [0]:
import tensorflow_datasets as tfds

In [0]:
imdb,info=tfds.load('imdb_reviews',with_info=True,as_supervised=True)

In [0]:
Training_data,Test_data=imdb['train'],imdb['test']

In [0]:
Training_Sentences=[]
Training_labels=[]



In [32]:
type(Training_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [0]:
for s,l in Training_data:
  Training_Sentences.append(str(s.numpy()))
  Training_labels.append(l.numpy())


In [34]:
Training_Sentences[0]

'b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."'

In [35]:
Training_labels[0]

0

In [0]:
Test_sentences=[]
Test_Labels=[]

for s,l in Test_data:
  Test_sentences.append(str(s.numpy()))
  Test_Labels.append(l.numpy())

In [37]:
print('Number of Training Examples:',len(Training_Sentences))
print('Number of Test Examples:',len(Test_sentences))


Number of Training Examples: 25000
Number of Test Examples: 25000


In [38]:
type(Training_labels)

list

In [0]:
import numpy as np

In [0]:
Training_labels_final=np.array(Training_labels)
Test_Labels_Final=np.array(Test_Labels)

In [41]:
Training_labels_final

array([0, 0, 0, ..., 0, 0, 1])

In [0]:
from tensorflow.keras.preprocessing.text import  Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
vocab_size=10000
oov_token='<OOV>'


In [0]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_token)

In [0]:
tokenizer.fit_on_texts(Training_Sentences)

In [0]:
Word_Index=tokenizer.word_index

In [0]:
print('Word_Index:',Word_Index)
print('Nmber of Unique Words:',len(Word_Index))

In [0]:
Training_Sequences=tokenizer.texts_to_sequences(Training_Sentences)

In [0]:
Training_Sequences[0]

In [0]:
Padded_sequences=pad_sequences(Training_Sequences,maxlen=120,truncating='post')

In [53]:
Padded_sequences[0]

array([   0,    0,   59,   12,   14,   35,  439,  400,   18,  174,   29,
          1,    9,   33, 1378, 3401,   42,  496,    1,  197,   25,   88,
        156,   19,   12,  211,  340,   29,   70,  248,  213,    9,  486,
         62,   70,   88,  116,   99,   24, 5740,   12, 3317,  657,  777,
         12,   18,    7,   35,  406, 8228,  178, 2477,  426,    2,   92,
       1253,  140,   72,  149,   55,    2,    1, 7525,   72,  229,   70,
       2962,   16,    1, 2880,    1,    1, 1506, 4998,    3,   40, 3947,
        119, 1608,   17, 3401,   14,  163,   19,    4, 1253,  927, 7986,
          9,    4,   18,   13,   14, 4200,    5,  102,  148, 1237,   11,
        240,  692,   13,   44,   25,  101,   39,   12, 7232,    1,   39,
       1378,    1,   52,  409,   11,   99, 1214,  874,  145,   10],
      dtype=int32)

In [0]:
Test_Sequences=tokenizer.texts_to_sequences(Test_sentences)

In [0]:
Pad_Test_Sequences=pad_sequences(Test_Sequences,maxlen=120,truncating='post')

In [0]:
Test_Sequences[0]

In [0]:
model=tf.keras.models.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,16,input_length=120),
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(6,activation='relu'),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
])

In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [63]:
num_epochs = 10
model.fit(Padded_sequences,Training_labels_final,epochs=num_epochs,validation_data=(Pad_Test_Sequences,Test_Labels_Final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 262us/sample - loss: 0.5046 - accuracy: 0.7295 - val_loss: 0.3871 - val_accuracy: 0.8254
Epoch 2/10
25000/25000 [==============================] - 5s 214us/sample - loss: 0.2363 - accuracy: 0.9106 - val_loss: 0.4194 - val_accuracy: 0.8143
Epoch 3/10
25000/25000 [==============================] - 6s 229us/sample - loss: 0.0834 - accuracy: 0.9787 - val_loss: 0.5271 - val_accuracy: 0.8027
Epoch 4/10
25000/25000 [==============================] - 5s 217us/sample - loss: 0.0208 - accuracy: 0.9974 - val_loss: 0.6062 - val_accuracy: 0.8056
Epoch 5/10
25000/25000 [==============================] - 5s 208us/sample - loss: 0.0061 - accuracy: 0.9994 - val_loss: 0.6798 - val_accuracy: 0.8080
Epoch 6/10
25000/25000 [==============================] - 5s 206us/sample - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.7398 - val_accuracy: 0.8058
Epoch 7/10
25000/25000 [==========================

In [64]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [71]:
reverse_word_index = dict([(value, key) for (key, value) in Word_Index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(Padded_sequences[1]))
print(Training_Sentences[1])

? ? ? ? ? ? ? b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all '
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of 

In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')